In [19]:
import data
import eval
import model
import preprocess
import postprocess

In [20]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

ONLY RUN IF THERE IS NEW DATA, TAKES LONG, NEXT CELL LOADS DATA FROM CSV AND PKL FILE

First off, the steps taken to fully load in the data set.

Step 1: load the raw text files and their id's into a dataframe

Step 2: if the texts are not in English, translate them

Step 3: concatenate the text files into a single dataset

Step 4: load in the labels

Step 5: merge the text and labels data sets

This can take quite a while, due to the translation step, skip this for the next cell, once we have all the data correctly saved as a csv, so we just load in that csv again. 

In [22]:
# load english text data
df_text_en = data.load_text_data("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/EN/raw-documents")

# load and translate hindi and portugese text
df_text_hi = data.load_text_data("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/HI/raw-documents", True, source_lang="hi")
df_text_pt = data.load_text_data("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/PT/raw-documents", True, source_lang = "pt")
df_text = pd.concat([df_text_en, df_text_hi, df_text_pt])

# load all the labels, these are all in english
df_labels_en = data.load_labels("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/EN/subtask-2-annotations.txt")
df_labels_hi = data.load_labels("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/HI/subtask-2-annotations.txt")
df_labels_pt = data.load_labels("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/PT/subtask-2-annotations.txt")
df_labels = pd.concat([df_labels_en, df_labels_hi, df_labels_pt])


dom_mlb, sub_mlb, dom_narr_enc, sub_narr_enc, df = preprocess.encode_labels(pd.merge(df_text, df_labels, on="id"))

# save the mlb's, and the dataframe, so we can read them instead of processing the entire data again
with open("dom_mlb.pkl", "wb") as f:
    pickle.dump(dom_mlb,f)

with open("sub_mlb.pkl", "wb") as f:
    pickle.dump(sub_mlb,f)

df.to_csv("data.csv")

Data length:  200
Error while reading:HI_82.txt
Error while reading:HI_55.txt
Error while reading:HI_115.txt
Error while reading:HI_128.txt
Error while reading:HI_100.txt
Error while reading:HI_40.txt
Error while reading:HI_54.txt
Error while reading:HI_81.txt
Error while reading:HI_42.txt
Error while reading:HI_116.txt
Error while reading:HI_56.txt
Error while reading:HI_117.txt
Error while reading:HI_94.txt
Error while reading:HI_80.txt
Error while reading:HI_84.txt
Error while reading:HI_113.txt
Error while reading:HI_53.txt
Error while reading:HI_112.txt
Error while reading:HI_46.txt
Error while reading:HI_91.txt
Error while reading:HI_85.txt
Error while reading:HI_93.txt
Error while reading:HI_87.txt


KeyboardInterrupt: 

first load the data set, and save it as a csv?

In [25]:
df = pd.read_csv("data.csv")

# load the mlb files back in, to get the classes and transform functions
with open("dom_mlb.pkl", "rb") as f:
    dom_mlb = pickle.load(f)

with open("sub_mlb.pkl", "rb") as f:
    sub_mlb = pickle.load(f)

print(df.head())
print(dom_mlb.classes_)
print(sub_mlb.classes_)

   Unnamed: 0                id  \
0           0  EN_UA_104876.txt   
1           1  EN_UA_023211.txt   
2           2  EN_UA_011260.txt   
3           3  EN_UA_101067.txt   
4           4  EN_UA_102963.txt   

                                                text  \
0  Putin honours army unit blamed for Bucha massa...   
1  Europe Putin thanks US journalist Tucker Carls...   
2  Russia has a clear plan to resolve the conflic...   
3  First war of TikTok era sees tragedy, humor an...   
4  Ukraine's President Zelenskyy to address Mexic...   

                                            dom_narr  \
0                                          ['Other']   
1                                          ['Other']   
2  ['URW: Russia is the Victim', 'URW: Discrediti...   
3                                          ['Other']   
4                                          ['Other']   

                                            sub_narr  \
0                                          ['Other']   
1  

Train it on the sub classes

In [26]:
vec = TfidfVectorizer(min_df=0.02,max_df=0.75)
X = vec.fit_transform(df.text)
y = df[sub_mlb.classes_].values # omg this works
print(y)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [27]:
y_pred, y_true = model.logistic_regression_loocv(X,y,majority_ensemble=True)
print(classification_report(y_true = y_true, y_pred=y_pred, target_names=sub_mlb.classes_))

Skipped fold 17, labels invalid
Skipped fold 51, labels invalid
Skipped fold 80, labels invalid
Skipped fold 89, labels invalid
Skipped fold 145, labels invalid
Skipped fold 154, labels invalid
Skipped fold 165, labels invalid
Skipped fold 178, labels invalid
Skipped fold 240, labels invalid
Skipped fold 241, labels invalid
Skipped fold 375, labels invalid
Skipped fold 383, labels invalid
Skipped fold 423, labels invalid
                                                                                                            precision    recall  f1-score   support

                                 CC: Amplifying Climate Fears: Amplifying existing fears of global warming       0.57      0.69      0.62        35
                                               CC: Amplifying Climate Fears: Doomsday scenarios for humans       0.33      0.55      0.41        11
                                            CC: Amplifying Climate Fears: Earth will be uninhabitable soon       1.00      0.60   

/Users/jochem/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jochem/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Trained on the dom classes:

In [28]:
y = df[dom_mlb.classes_].values
y_pred, y_true = model.logistic_regression_loocv(X,y,majority_ensemble=True)
print(classification_report(y_true = y_true, y_pred=y_pred, target_names=dom_mlb.classes_))

Skipped fold 165, labels invalid
                                                        precision    recall  f1-score   support

                          CC: Amplifying Climate Fears       0.62      0.84      0.72        58
                      CC: Climate change is beneficial       0.00      0.00      0.00         2
              CC: Controversy about green technologies       0.50      0.17      0.25         6
                     CC: Criticism of climate movement       0.41      0.57      0.48        21
                     CC: Criticism of climate policies       0.39      0.59      0.47        27
         CC: Criticism of institutions and authorities       0.55      0.78      0.64        49
                        CC: Downplaying climate change       1.00      0.22      0.36         9
       CC: Green policies are geopolitical instruments       0.00      0.00      0.00         0
 CC: Hidden plots by secret schemes of powerful groups       0.00      0.00      0.00         9
      

/Users/jochem/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jochem/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
